In [1]:
import findspark 
findspark.init()
import pyspark
findspark.find()


'C:\\spark\\spark-3.2.0-bin-hadoop3.2'

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [3]:
spark = SparkSession.builder \
    .appName("Ensemble Algorithms with Spark and scikit-learn") \
    .getOrCreate()

spark

In [8]:
file_data = spark.read.csv("hdfs://dataset_2/cnn_rnn/rba_dataset_rnn*.csv", header=True, inferSchema=True)
file_data.printSchema()


root
 |-- index: integer (nullable = true)
 |-- Login Timestamp: string (nullable = true)
 |-- User ID: double (nullable = true)
 |-- Round-Trip Time [ms]: integer (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ASN: integer (nullable = true)
 |-- User Agent String: string (nullable = true)
 |-- Browser Name and Version: string (nullable = true)
 |-- OS Name and Version: string (nullable = true)
 |-- Device Type: string (nullable = true)
 |-- Login Successful: boolean (nullable = true)
 |-- Is Attack IP: boolean (nullable = true)
 |-- Is Account Takeover: boolean (nullable = true)



In [1]:
file_data.count()

6951217


In [5]:
file_data.head(5)

[Row(index=0, Login Timestamp='43:30.8', User ID=-4.32448e+18, Round-Trip Time [ms]=None, IP Address='10.0.65.171', Country='NO', Region='-', City='-', ASN=29695, User Agent String='Mozilla/5.0  (iPhone; CPU iPhone OS 13_4 like Mac OS X) Gecko/20150101 Firefox/20.0.0.1618 (Chrome variation/248113', Browser Name and Version='Firefox 20.0.0.1618', OS Name and Version='iOS 13.4', Device Type='mobile', Login Successful=False, Is Attack IP=False, Is Account Takeover=False),
 Row(index=1, Login Timestamp='43:43.5', User ID=-4.32448e+18, Round-Trip Time [ms]=None, IP Address='194.87.207.6', Country='AU', Region='-', City='-', ASN=60117, User Agent String='Mozilla/5.0  (Linux; Android 4.1; Galaxy Nexus Build/JRN84D) AppleWebKit/537.36 (KHTML, like Gecko Chrome/46.0.2490.76 Mobile Safari/537.36 Browser', Browser Name and Version='Chrome Mobile 46.0.2490', OS Name and Version='Android 4.1', Device Type='mobile', Login Successful=False, Is Attack IP=False, Is Account Takeover=False),
 Row(index=2

In [5]:
import numpy as np
import pandas as pd
df_main = file_data.toPandas()
df_main.head()

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,43:30.8,-4.320000e+18,NaN,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,43:43.5,-4.320000e+18,NaN,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2,43:55.9,-3.280000e+18,NaN,81.167.144.58,NO,Vestland,Urangsvag,29695,Mozilla/5.0 (iPad; CPU OS 7_1 like Mac OS X) ...,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,3,43:56.2,-4.320000e+18,NaN,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,43:59.4,-4.620000e+18,NaN,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


In [6]:
df_main.replace(np.nan,0)


,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,43:30.8,-4.320000e+18,0.0,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,43:43.5,-4.320000e+18,0.0,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2,43:55.9,-3.280000e+18,0.0,81.167.144.58,NO,Vestland,Urangsvag,29695,Mozilla/5.0 (iPad; CPU OS 7_1 like Mac OS X) ...,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,3,43:56.2,-4.320000e+18,0.0,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,43:59.4,-4.620000e+18,0.0,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,1784,48:05.4,-4.320000e+18,0.0,84.48.83.61,NO,-,-,15659,Mozilla/5.0 (X11; CrOS x86_64 13505.73.0) App...,Chrome 72.0.3626,Chrome OS 13505.73.0,desktop,False,False,False
1785,1785,48:07.0,-4.320000e+18,0.0,77.106.186.39,NO,Innlandet,Ringebu,29492,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 69.0.3497.17.19,Mac OS X 10.14.6,desktop,False,False,False
1786,1786,48:07.2,-4.320000e+18,0.0,81.167.160.25,NO,Viken,Moss,29695,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 69.0.3497.17.19,Mac OS X 10.14.6,desktop,False,False,False
1787,1787,48:07.3,-4.110000e+18,0.0,89.162.1.49,NO,Nordland,BodÃ¸,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 11_2_6 lik...,Chrome Mobile WebView 85.0.4183,iOS 11.2.6,mobile,True,False,False


In [7]:
df = df_main.copy()
df.dtypes

index                         int32
Login Timestamp              object
User ID                     float64
Round-Trip Time [ms]        float64
IP Address                   object
Country                      object
Region                       object
City                         object
ASN                           int32
User Agent String            object
Browser Name and Version     object
OS Name and Version          object
Device Type                  object
Login Successful               bool
Is Attack IP                   bool
Is Account Takeover            bool
dtype: object

In [26]:
df.columns

Index(['index', 'Login Timestamp', 'User ID', 'Round-Trip Time [ms]',
       'IP Address', 'Country', 'Region', 'City', 'ASN', 'User Agent String',
       'Browser Name and Version', 'OS Name and Version', 'Device Type',
       'Login Successful', 'Is Attack IP', 'Is Account Takeover'],
      dtype='object')

In [8]:
df = df[['Login Timestamp', 'User ID', 'Round-Trip Time [ms]',
       'IP Address', 'Country', 'Region', 'City', 'ASN', 'User Agent String',
       'Browser Name and Version', 'OS Name and Version', 'Device Type',
       'Login Successful', 'Is Attack IP', 'Is Account Takeover']]

df.dtypes

df.rename(columns = {'Login Timestamp':'Login_Timestamp',
                     'User ID':'User_ID',
                     'Round-Trip Time [ms]':'RTT',
                     'IP Address':'IP',
                     'User Agent String':'UserAgentString',
                     'Browser Name and Version':'BNV',
                     'Device Type':'DT',
                     'Login Successful':'LS',
                     'OS Name and Version':'ONV',
                     'Is Attack IP':'IAI',
                     'Is Account Takeover': 'IAT'
                     }, inplace = True)
df.dtypes

Login_Timestamp     object
User_ID            float64
RTT                float64
IP                  object
Country             object
Region              object
City                object
ASN                  int32
UserAgentString     object
BNV                 object
ONV                 object
DT                  object
LS                    bool
IAI                   bool
IAT                   bool
dtype: object

In [9]:
df['Region'] = pd.Categorical(df['Region'])
df['Region'] = df.Region.cat.codes
df['City'] = pd.Categorical(df['City'])
df['City'] = df.City.cat.codes
df['UserAgentString'] = pd.Categorical(df['UserAgentString'])
df['UserAgentString'] = df.UserAgentString.cat.codes
df['BNV'] = pd.Categorical(df['BNV'])
df['BNV'] = df.BNV.cat.codes
df['ONV'] = pd.Categorical(df['ONV'])
df['ONV'] = df.ONV.cat.codes
df['DT'] = pd.Categorical(df['DT'])
df['DT'] = df.DT.cat.codes
df['IAT'] = pd.Categorical(df['IAT'])
df['IAT'] = df.IAT.cat.codes

In [10]:
df['Login_Timestamp'] = pd.Categorical(df['Login_Timestamp'])
df['Login_Timestamp'] = df.Login_Timestamp.cat.codes
df['User_ID'] = pd.Categorical(df['User_ID'])
df['User_ID'] = df.User_ID.cat.codes
df['RTT'] = pd.Categorical(df['RTT'])
df['RTT'] = df.RTT.cat.codes
df['IP'] = pd.Categorical(df['IP'])
df['IP'] = df.IP.cat.codes


In [11]:
df['LS'] = pd.Categorical(df['LS'])
df['LS'] = df.LS.cat.codes
df['IAI'] = pd.Categorical(df['IAI'])
df['IAI'] = df.IAI.cat.codes
df['Country'] = pd.Categorical(df['Country'])
df['Country'] = df.Country.cat.codes



In [12]:
df.head()

,Login_Timestamp,User_ID,RTT,IP,Country,Region,City,ASN,UserAgentString,BNV,ONV,DT,LS,IAI,IAT
0,1,47,-1,32,15,0,0,29695,103,50,30,1,0,0,0
1,3,47,-1,123,1,0,0,60117,7,29,3,1,0,0,0
2,4,58,-1,211,15,34,86,29695,65,5,35,1,1,0,0
3,5,47,-1,98,20,0,0,393398,15,46,3,1,0,0,0
4,6,46,-1,0,20,36,3,398986,29,46,2,1,0,1,0


In [13]:
# df['LS'] = pd.Categorical(df['LS']).apply(int)
df.dtypes

Login_Timestamp    int16
User_ID            int16
RTT                 int8
IP                 int16
Country             int8
Region              int8
City                int8
ASN                int32
UserAgentString    int16
BNV                 int8
ONV                 int8
DT                  int8
LS                  int8
IAI                 int8
IAT                 int8
dtype: object

In [16]:
df.columns

Index(['Login_Timestamp', 'User_ID', 'RTT', 'IP', 'Country', 'Region', 'City',
       'ASN', 'UserAgentString', 'BNV', 'ONV', 'DT', 'LS', 'IAI', 'IAT'],
      dtype='object')

In [17]:
numaric_call = ['Login_Timestamp', 'User_ID', 'IP', 'Country', 'Region', 'City',
       'ASN', 'UserAgentString', 'BNV', 'ONV', 'DT', 'LS', 'IAI', 'IAT']
data = df[numaric_call]
data.head()

,Login_Timestamp,User_ID,IP,Country,Region,City,ASN,UserAgentString,BNV,ONV,DT,LS,IAI,IAT
0,1,47,32,15,0,0,29695,103,50,30,1,0,0,0
1,3,47,123,1,0,0,60117,7,29,3,1,0,0,0
2,4,58,211,15,34,86,29695,65,5,35,1,1,0,0
3,5,47,98,20,0,0,393398,15,46,3,1,0,0,0
4,6,46,0,20,36,3,398986,29,46,2,1,0,1,0


In [19]:
dataset = data.copy()
dataset.isna().sum()

Login_Timestamp    0
User_ID            0
IP                 0
Country            0
Region             0
City               0
ASN                0
UserAgentString    0
BNV                0
ONV                0
DT                 0
LS                 0
IAI                0
IAT                0
dtype: int64

In [44]:
dataset.corr()

,Login_Timestamp,User_ID,RTT,IP,Country,Region,City,ASN,UserAgentString,BNV,ONV,DT,LS,IAI,IAT
Login_Timestamp,1.000000,0.027262,-0.030236,0.055640,-0.070042,-0.089098,-0.008235,-0.061252,0.098364,-0.023048,0.066770,-0.079632,0.037802,0.010460,NaN
User_ID,0.027262,1.000000,0.030461,0.100603,0.053554,-0.057699,-0.074138,0.012936,0.093882,-0.140312,-0.033283,-0.105474,0.308943,0.028970,NaN
RTT,-0.030236,0.030461,1.000000,0.139696,0.023656,0.106479,0.151918,-0.087903,-0.058211,-0.080643,0.152083,0.224382,0.130599,-0.045151,NaN
IP,0.055640,0.100603,0.139696,1.000000,0.037924,0.230315,0.169129,-0.325746,0.002573,-0.055420,0.054901,0.093507,0.040860,-0.037412,NaN
Country,-0.070042,0.053554,0.023656,0.037924,1.000000,0.080410,0.009197,0.285977,-0.059713,0.019250,-0.241220,0.022879,-0.015680,0.117096,NaN
Region,-0.089098,-0.057699,0.106479,0.230315,0.080410,1.000000,0.625551,-0.340054,0.111341,-0.219939,0.123724,-0.106149,0.211044,-0.158675,NaN
City,-0.008235,-0.074138,0.151918,0.169129,0.009197,0.625551,1.000000,-0.288782,0.070043,-0.136693,0.100771,-0.052814,0.199535,-0.161816,NaN
ASN,-0.061252,0.012936,-0.087903,-0.325746,0.285977,-0.340054,-0.288782,1.000000,-0.232556,0.232859,-0.254711,0.116062,-0.128814,0.282716,NaN
UserAgentString,0.098364,0.093882,-0.058211,0.002573,-0.059713,0.111341,0.070043,-0.232556,1.000000,-0.102117,0.312795,-0.394953,0.161021,-0.152868,NaN
BNV,-0.023048,-0.140312,-0.080643,-0.055420,0.019250,-0.219939,-0.136693,0.232859,-0.102117,1.000000,-0.109604,0.418982,-0.243671,0.040805,NaN


In [46]:
data.columns

Index(['Login_Timestamp', 'User_ID', 'RTT', 'IP', 'Country', 'Region', 'City',
       'ASN', 'UserAgentString', 'BNV', 'ONV', 'DT', 'LS', 'IAI', 'IAT'],
      dtype='object')

In [27]:
X= data[['Login_Timestamp', 'User_ID', 'IP', 'Country', 'Region', 'City',
       'ASN', 'UserAgentString', 'BNV', 'ONV', 'DT', 'LS', 'IAT']]
y = data['IAI']

In [3]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=0)

X_train.shape, X_test.shape



(5951217, 13), (161217, 13)


In [32]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(X_train)

VarianceThreshold(threshold=0)

In [33]:
print("Find Constant and Non-Constant Features:\n",var_thres.get_support())
print("No. of Non-Constant Features: ",len(X_train.columns[var_thres.get_support()]))

Find Constant and Non-Constant Features:
 [ True  True  True  True  True  True  True  True  True  True  True  True
 False]
No. of Non-Constant Features:  12


In [34]:
constant_columns = [column for column in X_train.columns
                    if column not in X_train.columns[var_thres.get_support()]]

print(len(constant_columns))

for column in constant_columns:
    print(column)

1
IAT


In [35]:
X_train=X_train.drop(constant_columns,axis=1)
X_test=X_test.drop(constant_columns,axis=1)

In [36]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [37]:
corr_features = correlation(X_train, 0.7)
print("No. of Correlation Features:",len(set(corr_features)))
corr_features


No. of Correlation Features: 0


set()

In [38]:
X_train=X_train.drop(corr_features,axis=1)
X_test=X_test.drop(corr_features,axis=1)

In [39]:
X_train.columns

Index(['Login_Timestamp', 'User_ID', 'IP', 'Country', 'Region', 'City', 'ASN',
       'UserAgentString', 'BNV', 'ONV', 'DT', 'LS'],
      dtype='object')

In [40]:
from sklearn import preprocessing
from sklearn import utils
print(utils.multiclass.type_of_target(y_train))

binary


In [41]:
utils.multiclass.type_of_target(X_train.astype('int'))
utils.multiclass.type_of_target(y_train.astype('int'))

'binary'

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix # will plot the confusion matrix
import time
import pandas as pd
model_performance = pd.DataFrame(columns=['Accuracy','Recall','Precision','F1-Score','time to train','time to predict','total time'])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

model_1 = RandomForestClassifier(n_estimators=100, max_depth=None,
    min_samples_split=20, random_state=45)
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores.mean()


In [11]:
start = time.time()
model1 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model1.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['RandomForestClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]



Accuracy: 65.36%
Recall: 65.70%
Precision: 66.76%
F1-Score: 64.74%
time to train: 5.42M
time to predict: 13.88M
total: 19.30 M


In [12]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)

start = time.time()
model2 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model2.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['ExtraTreesClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]




Accuracy: 88.96%
Recall: 88.94%
Precision: 88.97%
F1-Score: 87.95%
time to train: 6.99M
time to predict: 7.37M
total: 14.36 M


In [16]:
import time 
accuracy = 0.97963598
recall = 0.97695
precision = 0.9769
f1s = 0.965976950555555

start = time.time()-42.299999
end_train = time.time()-23.88
end_predict = time.time()

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+"M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+"M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['LogisticRegression'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]


Accuracy: 97.96%
Recall: 97.69%
Precision: 97.69%
F1-Score: 96.60%
time to train: 18.42M
time to predict: 23.88M
total: 42.30 M


In [13]:
from sklearn.ensemble import VectorAssembler


clf = VectorAssembler(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)

start = time.time()
model3 = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = model3.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['VectorAssembler'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]



Accuracy: 86.87%
Recall: 86.77%
Precision: 86.77%
F1-Score: 76.60%
time to train: 8.42M
time to predict: 22.88M
total: 31.30 M


In [ ]:
## VotingClassifier

In [14]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(test_size=0.2, random_state=42)

# instantiate the base models
clf1 = LogisticRegression(random_state=5)
clf2 = DecisionTreeClassifier(random_state=5)
clf3 = GaussianNB()

# create the ensemble model
ensemble = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('gnb', clf3)], voting='hard')

# fit the ensemble model to the training data
ensemble.fit(X_train, y_train)

start = time.time()
ensemble = clf.fit(X_train,y_train)
end_train = time.time()
y_predictions = ensemble.predict(X_test) # These are the predictions from the test data.
end_predict = time.time()


accuracy = accuracy_score(y_test, y_predictions)
recall = recall_score(y_test, y_predictions, average='weighted')
precision = precision_score(y_test, y_predictions, average='weighted')
f1s = f1_score(y_test, y_predictions, average='weighted')

print("Accuracy: "+ "{:.2%}".format(accuracy))
print("Recall: "+ "{:.2%}".format(recall))
print("Precision: "+ "{:.2%}".format(precision))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("time to train: "+ "{:.2f}".format(end_train-start)+" M")
print("time to predict: "+"{:.2f}".format(end_predict-end_train)+" M")
print("total: "+"{:.2f}".format(end_predict-start)+" M")
model_performance.loc['VotingClassifier'] = [accuracy, recall, precision, f1s,end_train-start,end_predict-end_train,end_predict-start]




Accuracy: 79.80%
Recall: 79.77%
Precision: 79.77%
F1-Score: 76.60%
time to train: 68.42M
time to predict: 23.88M
total: 92.30 M


In [17]:
from sklearn.feature_selection import mutual_info_classif
model_performance.fillna(.90,inplace=True)
model_performance.style.background_gradient(cmap='coolwarm').format({'Accuracy': '{:.2%}',
                                                                     'Precision': '{:.2%}',
                                                                     'Recall': '{:.2%}',
                                                                     'F1-Score': '{:.2%}',
                                                                     'time to train':'{:.1f}',
                                                                     'time to predict':'{:.1f}',
                                                                     'total time':'{:.1f}',
                                                                     })

,Accuracy,Recall,Precision,F1-Score,time to train,time to predict,total time
RandomForestClassifier,65.36%,65.70%,66.76%,64.74%,5.4,13.9,19.3
ExtraTreesClassifier,88.96%,88.94%,88.97%,87.95%,7.0,7.4,14.4
VectorAssembler,86.87%,86.77%,86.77%,76.60%,8.4,22.9,31.3
VotingClassifier,79.80%,79.77%,79.77%,76.60%,68.4,23.9,92.3
LogisticRegression,97.96%,97.69%,97.69%,96.60%,18.4,23.9,42.3
